In [1]:
"""Суть задачи 🧬
У нас есть ДНК-последовательность, состоящая всего из 4 букв: A, C, G, T. 
В Python строка хранит каждый символ как Unicode, что занимает 8 байт = 64 бита на символ! 
Это дикая неэффективность, потому что для кодирования 4 вариантов достаточно всего 2 бит.

Математика эффективности:
4 варианта = 2² → нужно 2 бита
Обычная строка: 8 байт = 64 бита на символ
Сжатый вариант: 2 бита на символ
Экономия: (64-2)/64 × 100% = 96.875% (в книге считают 8 бит → 75%)
"""

'Суть задачи 🧬\nУ нас есть ДНК-последовательность, состоящая всего из 4 букв: A, C, G, T. \nВ Python строка хранит каждый символ как Unicode, что занимает 8 байт = 64 бита на символ! \nЭто дикая неэффективность, потому что для кодирования 4 вариантов достаточно всего 2 бит.\n\nМатематика эффективности:\n4 варианта = 2² → нужно 2 бита\nОбычная строка: 8 байт = 64 бита на символ\nСжатый вариант: 2 бита на символ\nЭкономия: (64-2)/64 × 100% = 96.875% (в книге считают 8 бит → 75%)\n'

In [2]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
# Что происходит: При создании объекта мы сразу сжимаем переданную строку с генами.

In [3]:
# Шаг 2: Сжатие — магия побитовых операций

def _compress(self, gene: str) -> None:
    self.bit_string = 1  # ← ХИТРОСТЬ: начальная метка!
    
    for nucleotide in gene.upper():
        self.bit_string <<= 2  # Сдвиг влево на 2 бита
        
        if nucleotide == "A":
            self.bit_string |= 0b00  # Добавляем 00
        elif nucleotide == "C":
            self.bit_string |= 0b01  # Добавляем 01
        elif nucleotide == "G":
            self.bit_string |= 0b10  # Добавляем 10
        elif nucleotide == "T":
            self.bit_string |= 0b11  # Добавляем 11

In [4]:
"""
Почему bit_string = 1? 🤔
Это "стражник" (sentinel) - специальная метка, чтобы отличать начало последовательности. 
Без неё, если ген начинается с A (00), мы бы не знали, где реальные данные.

Пример без метки:

Кодируем "A" → 0b00 → это просто число 0

Как отличить "пустую последовательность" от "последовательности с A"? Никак!

С меткой:

Пусто: 0b1 (просто метка)

"A": сдвигаем 0b1 << 2 = 0b100, добавляем 00 → 0b100 (метка + A)

Побитовые операции в деталях 🔬
Возьмем строку "AT":

Шаг	Действие	bit_string (двоично)	bit_string (десятично)
0	Начало	1	1
1	Сдвиг << 2	100	4
2	Добавляем A (00) через OR	100	4
3	Сдвиг << 2	10000	16
4	Добавляем T (11) через OR	10011	19
Результат: "AT" = 19 в памяти!"""

'\nПочему bit_string = 1? 🤔\nЭто "стражник" (sentinel) - специальная метка, чтобы отличать начало последовательности. \nБез неё, если ген начинается с A (00), мы бы не знали, где реальные данные.\n\nПример без метки:\n\nКодируем "A" → 0b00 → это просто число 0\n\nКак отличить "пустую последовательность" от "последовательности с A"? Никак!\n\nС меткой:\n\nПусто: 0b1 (просто метка)\n\n"A": сдвигаем 0b1 << 2 = 0b100, добавляем 00 → 0b100 (метка + A)\n\nПобитовые операции в деталях 🔬\nВозьмем строку "AT":\n\nШаг\tДействие\tbit_string (двоично)\tbit_string (десятично)\n0\tНачало\t1\t1\n1\tСдвиг << 2\t100\t4\n2\tДобавляем A (00) через OR\t100\t4\n3\tСдвиг << 2\t10000\t16\n4\tДобавляем T (11) через OR\t10011\t19\nРезультат: "AT" = 19 в памяти!'

In [5]:
# Шаг 3: Распаковка - чтение задом наперед

def decompress(self) -> str:
    gene: str = ""
    # Проходим по битам справа налево с шагом 2
    for i in range(0, self.bit_string.bit_length() - 1, 2):
        bits: int = self.bit_string >> i & 0b11
        
        if bits == 0b00: gene += "A"
        elif bits == 0b01: gene += "C"
        elif bits == 0b10: gene += "G"
        elif bits == 0b11: gene += "T"
    
    return gene[::-1]  # Разворачиваем результат

In [6]:
"""
Почему читаем справа налево? 🔄
При сжатии мы добавляли новые биты справа (из-за сдвига влево и OR). 
Значит, при чтении самый последний нуклеотид находится в правых битах.

Пример с "AT": 0b10011

Читаем справа: 11 → T

Сдвигаемся левее: 100 → A

Получаем "TA", разворачиваем → "AT"


Визуализация работы 🎨

Исходная строка:    A     T     G
                    ↓     ↓     ↓
Двоичный код:      00    11    10
                    ↓     ↓     ↓
Битовая строка:    00  + 11  + 10
                    ↓
С меткой 1 в начале: 1 00 11 10
                    ↓
В памяти (int):    0b1001110 = 78
"""

'\nПочему читаем справа налево? 🔄\nПри сжатии мы добавляли новые биты справа (из-за сдвига влево и OR). \nЗначит, при чтении самый последний нуклеотид находится в правых битах.\n\nПример с "AT": 0b10011\n\nЧитаем справа: 11 → T\n\nСдвигаемся левее: 100 → A\n\nПолучаем "TA", разворачиваем → "AT"\n\n\nВизуализация работы 🎨\n\nИсходная строка:    A     T     G\n                    ↓     ↓     ↓\nДвоичный код:      00    11    10\n                    ↓     ↓     ↓\nБитовая строка:    00  + 11  + 10\n                    ↓\nС меткой 1 в начале: 1 00 11 10\n                    ↓\nВ памяти (int):    0b1001110 = 78\n'

In [7]:
"""
Сравнение производительности 📊
"""
original = "TAGGGATTA..." * 100  # Длинная строка
print(f"Оригинал: {getsizeof(original)} байт")
# Оригинал: 8649 байт

compressed = CompressedGene(original)
print(f"Сжато: {getsizeof(compressed.bit_string)} байт")
# Сжато: 2320 байт

print(f"Экономия: {(8649-2320)/8649*100:.1f}%")
# Экономия: 73.2%


NameError: name 'getsizeof' is not defined

In [ ]:
"""
Где это пригодится в реальной жизни? 🌍

1. Биоинформатика и геномика
Реальные геномы ОГРОМНЫ:

Ген человека: ~3.2 миллиарда нуклеотидов

Без сжатия: 3.2e9 × 8 байт = 25.6 ГБ

Со сжатием: 3.2e9 × 0.25 байт = 0.8 ГБ

Экономия: 24.8 ГБ на один геном!

2. Базы данных с ограниченным алфавитом
Цвета: RGB (256×256×256 = 16M цветов) можно кодировать в 24 бита вместо хранения как строки

Карты: Кодировка тайлов в играх (трава = 00, вода = 01, гора = 10, лес = 11)

QR-коды: Хранят данные в максимально плотной упаковке

3. Сетевое взаимодействие
Протоколы IoT: Умные датчики передают данные по радио с ограниченным бюджетом энергии — каждый бит на счету

Спутниковая связь: Стоимость передачи данных колоссальна, сжатие критично

4. Базы данных и кэши
Представьте: храним миллиарды записей с полом
Вместо строк "MALE"/"FEMALE" (4-6 байт)
Храним как биты: 0 = MALE, 1 = FEMALE
Экономия: 4 байта → 0.125 байта (в 32 раза!)
"""

In [ ]:
"""Подводные камни и нюансы ⚠️
1. Метка (sentinel) занимает место
Для коротких последовательностей метка может составлять значительную долю данных.

2. Python int — не настоящая битовая строка
В Python int — это объект с накладными расходами:"""

In [ ]:
x = 1  # всего 1 бит данных
print(getsizeof(x))  # 28 байт! (224 бита)

# Но для длинных последовательностей дополнительные биты добавляются эффективно.

In [ ]:
"""3. Скорость vs Память
Сжатие экономит память, но требует CPU для упаковки/распаковки. Это классический компромисс."""

In [ ]:
# 4. Читаемость кода

# Вариант с if/elif (из книги) — понятно, но громоздко
if bits == 0b00: gene += "A"
elif bits == 0b01: gene += "C"

# Вариант со словарем — элегантнее
nucleotides = {0b00: "A", 0b01: "C", 0b10: "G", 0b11: "T"}
gene += nucleotides[bits]